# Credit to [ragnar](https://www.kaggle.com/ragnar123). This notebook builds upon their data pipeline found in  [ragnar's notebook](https://www.kaggle.com/ragnar123/very-fst-model)

### Initial notebook:
1. Read in data from calendar, sell_prices, sales_train_validation, submission and apply common method to reduce the memory usage
2. Merge data into a multivariate time series (MTS) format with each entry in the series describing the target variable (demand) and its exogeneous variables in different dates for different item_id, dept_id, cat_id, store_id and state_id
3. Transform features into the MTS (fill NaN events with category 'unknown' and encode all categorical variables)
4. Engineer additional features based on the series of prices and demand for each good. Examples include aggregations over lagged sub-series of the two variables
### Personal contributions (thus far)
5. Hand-crafted parameters for the lightGBM regressor based upon [documentation](https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html)
6. Implemented hyper-parameter tuning pipeline for optimizing the n_estimators, max_depth, num_leaves, learning_rate
### Future plans
7. Find a way to read in the full dataset and avoid memory problems
8. Implement a feature selection method (sklearn.feature_selection)
9. Engineer additional features

In [1]:
# import libraries
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import json
import gc
import os
import sys

In [2]:
# common function to reduce the memory usage thus allowing us to work with larger datasets
def reduce_mem_usage(df, verbose=True):
    """
    Common function to reduce the size of the entries in a pandas DataFrame.
    """
    import numpy as np
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# simple function to read the data in the competition files
def readData(submission_only=False, PATH=''):
    import pandas as pd
    print('Reading files...')
    submission = pd.read_csv(PATH+'m5-forecasting-accuracy/sample_submission.csv')
    if submission_only:
        return submission
    else:
        calendar = pd.read_csv(PATH+'m5-forecasting-accuracy/calendar.csv')
        calendar = reduce_mem_usage(calendar)
        print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))

        sell_prices = pd.read_csv(PATH+'m5-forecasting-accuracy/sell_prices.csv')
        sell_prices = reduce_mem_usage(sell_prices)
        print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))

        sales_train_validation = pd.read_csv(PATH+'m5-forecasting-accuracy/sales_train_validation.csv')
        print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))

    return calendar, sell_prices, sales_train_validation, submission



In [3]:
# process the data to get it into a tabular format; pd.melt is especially useful to 'unpack' the target variable (demand)
def melt_and_merge():
    calendar, sell_prices, sales_train_validation, submission = readData()
    # drop some calendar features
    calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
    
    # melt sales data, get it ready for training
    sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                                     var_name = 'day', value_name = 'demand')
    
    #print('Melted sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    sales_train_validation = reduce_mem_usage(sales_train_validation)
    
    # seperate test dataframes
    test1_rows = [row for row in submission['id'] if 'validation' in row]
    #test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test1 = submission[submission['id'].isin(test1_rows)]
    #test2 = submission[submission['id'].isin(test2_rows)]
    
    # change column names
    test1.columns = ['id'] + ['d_{}'.format(i) for i in range(1914,1942)]
    #test2.columns = ['id'] + ['d_{}'.format(i) for i in range(1942,1970)]


    # get product table
    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    # merge with product table
    #test2['id'] = test2['id'].str.replace('_evaluation','_validation')
    test1 = test1.merge(product, how = 'left', on = 'id')
    #test2 = test2.merge(product, how = 'left', on = 'id')
    #test2['id'] = test2['id'].str.replace('_validation','_evaluation')
    
    # 
    test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day',
                    value_name = 'demand')
    #test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day',
    #                value_name = 'demand')
    
    sales_train_validation = pd.concat([sales_train_validation, test1], axis = 0) # include test2 later
    
    del test1#, test2
    gc.collect()
    
    # delete test2 for now
    #data = data[data['part'] != 'test2']
    
    sales_train_validation = pd.merge(sales_train_validation, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
    sales_train_validation.drop(['d', 'day'], inplace = True, axis = 1)
    
    # get the sell price data (this feature should be very important)
    sales_train_validation = sales_train_validation.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
    print('Our final dataset to train has {} rows and {} columns'.format(
        sales_train_validation.shape[0], sales_train_validation.shape[1]))
    
    del calendar, sell_prices; gc.collect();
    
    return sales_train_validation

In [4]:
# this function fills up the Nan values and encodes the categorical variables
def transform(data):
    from sklearn.preprocessing import LabelEncoder
    # convert to datetime object
    data['date'] = pd.to_datetime(data.date)
    
    # fill NaN features with unknown
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
    
    # Encode categorical features
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        encoder = LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
    
    return data


In [5]:
# this function computes useful laging features from the target variable and the price
# to convert what is initially a sequence to sequence mapping into a regression task (sequence to one),
# the author used lagged values starting from the minimum lag of 28 days, which is the forecasting horizon
def simple_fe(data):
    
    # rolling demand features
    data['lag_t28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
    data['lag_t29'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(29))
    data['lag_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(30))
    data['rolling_mean_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
    data['rolling_std_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
    data['rolling_mean_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
    data['rolling_mean_t90'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
    data['rolling_mean_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
    data['rolling_std_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
    data['rolling_skew_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).skew())
    data['rolling_kurt_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).kurt())
    
    
    # price features
    data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
    data['price_change_t1'] = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1'])
    data['rolling_price_max_t365'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())
    data['price_change_t365'] = (data['rolling_price_max_t365'] - data['sell_price']) / (data['rolling_price_max_t365'])
    data['rolling_price_std_t7'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
    data['rolling_price_std_t30'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std())
    data.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)
    
    # time features
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['week'] = data['date'].dt.week
    data['day'] = data['date'].dt.day
    data['dayofweek'] = data['date'].dt.dayofweek
    
    return data

In [6]:
new_fe = False
if new_fe:
    data = melt_and_merge()
    submission = readData(submission_only=True)
    data = transform(data)
    print('There are {:e} / {:e} NaN entries in the sell_price column'.format(data.sell_price.isna().sum(),data.shape[0]))
    data = simple_fe(data)
    data = reduce_mem_usage(data)
    data.to_pickle('engineered_data.pkl')
else:
    data = pd.read_pickle('engineered_data.pkl')
    submission = readData(submission_only=True)

Reading files...


### Apply hyperopt to select the parameters of the lightGBM
1. Define a loss function
2. Define the parameter space
3. Run Bayesian Optimization

In [7]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# define list of features
features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'year', 'month', 'week', 'day', 'dayofweek', 'event_name_1',
            'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_t28', 'lag_t29',
            'lag_t30', 'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30', 'rolling_mean_t90', 'rolling_mean_t180', 
            'rolling_std_t30', 'price_change_t1', 'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30', 
            'rolling_skew_t30', 'rolling_kurt_t30']

def optimize_parameters(x_train, x_val, max_evals=20):
    # define fixed hyperparameters
    params = {
        'tree_learner':'voting',
        'boosting_type': 'gbdt',
        'objective': 'tweedie',
        'tweedie_variance_power': 1.1,
        'metric': 'rmse',
        'subsample': 0.5,
        'subsample_freq': 1,
        'sub_feature' : 0.8,
        'sub_row' : 0.75,
        'bagging_freq' : 1,
        'lambda_l2' : 0.1,
        'verbosity': 1,
        'boost_from_average': True,
        'n_jobs': -1,
        'learning_rate':0.1,
        'seed': 3008,
        'verbose': -1}
    
    # define floating hyperparameters
    space = {
        'n_estimators': hp.quniform('n_estimators', 25, 600, 25),
        'max_depth': hp.quniform('max_depth', 1, 6, 1),
        'num_leaves': hp.quniform('num_leaves', 10, 120, 1),
    }
    
    # define the objective function to optimize the hyperparameters
    def objective(floating_params):
        params['n_estimators'] = int(floating_params['n_estimators'])
        params['max_depth'] = int(floating_params['max_depth'])
        params['num_leaves'] = int(floating_params['num_leaves'])

        print(params)
        regressor = lgb.LGBMRegressor(**params)
        
        x_sample = x_train.sample(int(x_train.shape[0]/50))
        x_train_sample, y_train_sample = x_sample[features], x_sample['demand']
        train_set = lgb.Dataset(x_train_sample[features], y_train_sample)
        
        model = lgb.train(params, train_set, num_boost_round = 2500)
        val_pred = model.predict(x_val[features])
        score = np.sqrt(mean_squared_error(val_pred, x_val['demand']))
        
        print("rmse {:.3f} params {}".format(score, params))
        return score

    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=max_evals)
    
    best['num_iterations'] =  1500
    best['n_estimators'] = int(best['n_estimators']); best['max_depth'] = int(best['max_depth']); 
    best['num_leaves'] = int(best['num_leaves']);
    
    with open('params.txt', 'w') as file:
        file.write(json.dumps(params))

def optimized_lgb(data, update_hyperparameters=False):
    
    # going to evaluate with the last 28 days
    x_train = data[data['date'] <= '2016-03-27']
    y_train = x_train['demand']
    x_val = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
    y_val = x_val['demand']
    test = data[(data['date'] > '2016-04-24')]
    del data
    gc.collect()
    
    if update_hyperparameters:
        optimize_parameters(x_train, x_val)
    

    train_set = lgb.Dataset(x_train[features], y_train)
    val_set = lgb.Dataset(x_val[features], y_val)
    del x_train, y_train
    gc.collect()
    

    with open('params.txt') as params_file:    
        params = json.load(params_file)
    
    model = lgb.train(params, train_set, num_boost_round = 2500, early_stopping_rounds = 50, 
                      verbose_eval = 100, valid_sets = [train_set, val_set])
    
    val_pred = model.predict(x_val[features])
    val_score = np.sqrt(mean_squared_error(val_pred, y_val))
    print(f'Our val rmse score is {val_score}')
    y_pred = model.predict(test[features])
    test['demand'] = y_pred
    
    return test

def append_predictions(test, submission):
    predictions = test[['id', 'date', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]

    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('submission.csv', index = False)

In [8]:
# eliminate the first 90 datapoints
if(data.shape[0]>5e7):
    data = data.sort_values('date').iloc[int(2.75e7):]
data = reduce_mem_usage(data)
test = optimized_lgb(data,update_hyperparameters=True)
append_predictions(test, submission)

Mem. usage decreased to 2386.86 Mb (0.0% reduction)
{'tree_learner': 'voting', 'boosting_type': 'gbdt', 'objective': 'tweedie', 'tweedie_variance_power': 1.1, 'metric': 'rmse', 'subsample': 0.5, 'subsample_freq': 1, 'sub_feature': 0.8, 'sub_row': 0.75, 'bagging_freq': 1, 'lambda_l2': 0.1, 'verbosity': 1, 'boost_from_average': True, 'n_jobs': -1, 'learning_rate': 0.1, 'seed': 3008, 'verbose': -1, 'n_estimators': 200, 'max_depth': 3, 'num_leaves': 119}
  0%|                                                                           | 0/20 [00:00<?, ?trial/s, best loss=?]

C:\Users\vlad-marius.griguta\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



rmse 2.203 params {'tree_learner': 'voting', 'boosting_type': 'gbdt', 'objective': 'tweedie', 'tweedie_variance_power': 1.1, 'metric': 'rmse', 'subsample': 0.5, 'subsample_freq': 1, 'sub_feature': 0.8, 'sub_row': 0.75, 'bagging_freq': 1, 'lambda_l2': 0.1, 'verbosity': 1, 'boost_from_average': True, 'n_jobs': -1, 'learning_rate': 0.1, 'seed': 3008, 'verbose': -1, 'n_estimators': 200, 'max_depth': 3, 'num_leaves': 119}
{'tree_learner': 'voting', 'boosting_type': 'gbdt', 'objective': 'tweedie', 'tweedie_variance_power': 1.1, 'metric': 'rmse', 'subsample': 0.5, 'subsample_freq': 1, 'sub_feature': 0.8, 'sub_row': 0.75, 'bagging_freq': 1, 'lambda_l2': 0.1, 'verbosity': 1, 'boost_from_average': True, 'n_jobs': -1, 'learning_rate': 0.1, 'seed': 3008, 'verbose': -1, 'n_estimators': 325, 'max_depth': 5, 'num_leaves': 107}
  5%|██▌                                               | 1/20 [00:08<02:48,  8.87s/trial, best loss: 2.202898480599774]

C:\Users\vlad-marius.griguta\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



rmse 2.150 params {'tree_learner': 'voting', 'boosting_type': 'gbdt', 'objective': 'tweedie', 'tweedie_variance_power': 1.1, 'metric': 'rmse', 'subsample': 0.5, 'subsample_freq': 1, 'sub_feature': 0.8, 'sub_row': 0.75, 'bagging_freq': 1, 'lambda_l2': 0.1, 'verbosity': 1, 'boost_from_average': True, 'n_jobs': -1, 'learning_rate': 0.1, 'seed': 3008, 'verbose': -1, 'n_estimators': 325, 'max_depth': 5, 'num_leaves': 107}
{'tree_learner': 'voting', 'boosting_type': 'gbdt', 'objective': 'tweedie', 'tweedie_variance_power': 1.1, 'metric': 'rmse', 'subsample': 0.5, 'subsample_freq': 1, 'sub_feature': 0.8, 'sub_row': 0.75, 'bagging_freq': 1, 'lambda_l2': 0.1, 'verbosity': 1, 'boost_from_average': True, 'n_jobs': -1, 'learning_rate': 0.1, 'seed': 3008, 'verbose': -1, 'n_estimators': 375, 'max_depth': 4, 'num_leaves': 25}
 10%|████▉                                            | 2/20 [00:25<03:20, 11.15s/trial, best loss: 2.1502202221841205]

...

C:\Users\vlad-marius.griguta\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



rmse 2.161 params {'tree_learner': 'voting', 'boosting_type': 'gbdt', 'objective': 'tweedie', 'tweedie_variance_power': 1.1, 'metric': 'rmse', 'subsample': 0.5, 'subsample_freq': 1, 'sub_feature': 0.8, 'sub_row': 0.75, 'bagging_freq': 1, 'lambda_l2': 0.1, 'verbosity': 1, 'boost_from_average': True, 'n_jobs': -1, 'learning_rate': 0.1, 'seed': 3008, 'verbose': -1, 'n_estimators': 275, 'max_depth': 2, 'num_leaves': 28}
100%|████████████████████████████████████████████████| 20/20 [04:37<00:00, 13.90s/trial, best loss: 2.1502202221841205]


C:\Users\vlad-marius.griguta\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 20 rounds.
[100]	training's rmse: 2.54277	valid_1's rmse: 2.20864
[200]	training's rmse: 2.50097	valid_1's rmse: 2.18099
Did not meet early stopping. Best iteration is:
[275]	training's rmse: 2.48489	valid_1's rmse: 2.16672
Our val rmse score is 2.166719020245881


C:\Users\vlad-marius.griguta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
